In [1]:
from pathlib import Path
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import torch.optim as optim
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.widgets import *
from cmd_classes_funcs_Marchese import *

/home/CAMPUS/cemb2020/anaconda3/envs/arcslab/lib/python3.8/site-packages/fastbook/__init__.py:18: UserWarning: Missing `graphviz` - please run `conda install fastbook`
  except ModuleNotFoundError: warn("Missing `graphviz` - please run `conda install fastbook`")


In [2]:
# Make sure we're running on the server's GPU
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=3)

In [3]:
# Get classes and filenames
path = Path("data")
classes = get_class_labels(path)
all_filenames = get_filenames(path)

## Splitting Data into train/validate sets

In [4]:
# Getting size of dataset and corresponding list of indices
dataset_size = len(all_filenames)
dataset_indices = list(range(dataset_size))

In [5]:
# Shuffling the indices
np.random.shuffle(dataset_indices)

In [6]:
# Getting index for where we want to split the data
val_split_index = int(np.floor(0.2 * dataset_size))

In [7]:
# Splitting list of indices into training and validation indices
train_idx, val_idx = dataset_indices[val_split_index:], dataset_indices[:val_split_index]

In [8]:
# Creating samplers
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [9]:
# Getting list of filenames for training and validation set
train_filenames = [all_filenames[i] for i in train_idx]
val_filenames = [all_filenames[i] for i in val_idx]

In [10]:
# Create training and validation datasets
train_data = ImageWithCmdDataset(classes, train_filenames)
val_data = ImageWithCmdDataset(classes, val_filenames)

In [11]:
train_loader = DataLoader(dataset=train_data, shuffle=False, batch_size=16, sampler=train_sampler)
val_loader = DataLoader(dataset=val_data, shuffle=False, batch_size=16, sampler=val_sampler)

In [12]:
# Instantiate MyModel class
net = MyModel_dnet201()

In [13]:
# Send model to GPU
net.to(device)

MyModel_dnet201(
  (cnn): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum

In [14]:
n = dataset_size
w = torch.tensor([(n-138)/n,(n-211)/n,(n-786)/n])
w = w.to(device)
# defining loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=w)
optimizer = optim.Adam(net.parameters(), lr=0.0000095)

In [15]:
num_epochs = 40

In [16]:
from time import time

In [ ]:
# Model training

net.train()

for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    
    start = time()
    
    for data in train_loader:
        # Get the inputs and labels
        inp_data, label = data
        
        # Break up the inputs
        img, cmd = inp_data
        
        # Putting data into the GPU
        img = img.to(device)
        cmd = cmd.to(device)
        label = label.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        output = net((img, cmd))
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print(f"Epoch:{epoch+1:}/{num_epochs}, Training Loss:{running_loss:0.1f}, Time:{time()-start:0.1f}s")

print('Finished Training')

Epoch:1/40, Training Loss:259.2, Time:198.9s


In [ ]:
# Checking accuracy on validation set

correct = 0
total = 0

# Variables to keep track of accuracy for each class
class_correct = [0 for _ in classes]
class_total = [0 for _ in classes]

net.eval()

with torch.no_grad():

    for data in val_loader:

        # Get the inputs and label data
        inp_data, label = data
        
        # Break up the inputs
        img, cmd = inp_data
        
        # Putting data into the GPU
        img = img.to(device)
        cmd = cmd.to(device)
        label = label.to(device)


        # Predict
        output = net((img, cmd))
        
        # Assuming we always get batches
        for i in range(output.size()[0]):
                
            # Getting the predicted most probable move
            move = torch.argmax(output[i])
                
            if move == label[i]:
                class_correct[label[i]] += 1
                class_total[label[i]] += 1
                correct +=1
            else:
                class_total[label[i]] += 1
            total += 1
        
# Calculate and output total set accuracy 
accuracy = correct / total
print(f"Accuracy on validation set: {correct}/{total} = {accuracy*100:.2f}%")

# Calculate and show accuracy for each class
for i, cls in enumerate(classes):
    ccorrect = class_correct[i]
    ctotal = class_total[i]
    caccuracy = ccorrect / ctotal
    print(f"  Accuracy on {cls:>5} class: {ccorrect}/{ctotal} = {caccuracy*100:.2f}%")

In [ ]:
PATH = 'cmd_torch_dnet201.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
data, labels = next(iter(val_loader))
labels.shape

In [ ]:
plt.imshow(data[0][0].permute(1,2,0))

In [ ]:
labels[0]

In [ ]:
net.eval()
img = data[0][0]
cmd = data[1][0]
img = img.to(device)
cmd = cmd.to(device)
net((img.unsqueeze(0), cmd.unsqueeze(0)))